# CatBoost Algorithms Part 2

CatBoost (categorical boosting) is an algorithm for gradient boosting on decision trees. Is a machine learning technique created by Yandex; therefore, it outperforms many existing boosting such as XGBoost and Light GBM. 

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import yfinance as yf
yf.pdr_override()

In [2]:
# input
symbol = 'AMD'
start = '2007-01-01'
end = '2018-11-16'

# Read data 
dataset = yf.download(symbol,start,end)

# View Columns
dataset.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2007-01-03 00:00:00-05:00,20.080000,20.400000,19.350000,19.520000,19.520000,28350300
2007-01-04 00:00:00-05:00,19.660000,19.860001,19.320000,19.790001,19.790001,23652500
2007-01-05 00:00:00-05:00,19.540001,19.910000,19.540001,19.709999,19.709999,15902400
2007-01-08 00:00:00-05:00,19.709999,19.860001,19.370001,19.469999,19.469999,15814800
2007-01-09 00:00:00-05:00,19.450001,19.709999,19.370001,19.650000,19.650000,14494200


In [3]:
dataset.index = pd.Index(dataset.index.date, name="Date")
dataset

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2007-01-03,20.080000,20.400000,19.350000,19.520000,19.520000,28350300
2007-01-04,19.660000,19.860001,19.320000,19.790001,19.790001,23652500
2007-01-05,19.540001,19.910000,19.540001,19.709999,19.709999,15902400
2007-01-08,19.709999,19.860001,19.370001,19.469999,19.469999,15814800
2007-01-09,19.450001,19.709999,19.370001,19.650000,19.650000,14494200
...,...,...,...,...,...,...
2018-11-09,20.770000,21.190001,20.110001,21.030001,21.030001,85900700
2018-11-12,20.680000,20.850000,18.799999,19.030001,19.030001,95948200
2018-11-13,19.280001,20.020000,18.969999,19.610001,19.610001,76126000


In [4]:
dataset['Increase_Decrease'] = np.where(dataset['Volume'].shift(-1) > dataset['Volume'],1,0)
dataset['Buy_Sell_on_Open'] = np.where(dataset['Open'].shift(-1) > dataset['Open'],1,0)
dataset['Buy_Sell'] = np.where(dataset['Adj Close'].shift(-1) > dataset['Adj Close'],1,0)
dataset['Returns'] = dataset['Adj Close'].pct_change()
dataset = dataset.dropna()
dataset.head()

,Open,High,Low,Close,Adj Close,Volume,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Returns
Date,,,,,,,,,,
2007-01-04,19.660000,19.860001,19.320000,19.790001,19.790001,23652500,0,0,0,0.013832
2007-01-05,19.540001,19.910000,19.540001,19.709999,19.709999,15902400,0,1,0,-0.004043
2007-01-08,19.709999,19.860001,19.370001,19.469999,19.469999,15814800,0,0,1,-0.012177
2007-01-09,19.450001,19.709999,19.370001,19.650000,19.650000,14494200,1,1,1,0.009245
2007-01-10,19.639999,20.020000,19.500000,20.010000,20.010000,19783200,1,1,1,0.018321


In [5]:
dataset.shape

(2990, 10)

In [6]:
X = dataset[['Open', 'High', 'Low', 'Volume']].values
y = dataset['Buy_Sell'].values

In [7]:
import catboost as cb
from catboost import CatBoostClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
# from sklearn.model_selection import GridSearchCV

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [9]:
cb = CatBoostClassifier()
cb.fit(X_train, y_train)

Learning rate set to 0.014951
0:	learn: 0.6929853	total: 142ms	remaining: 2m 21s
1:	learn: 0.6928677	total: 144ms	remaining: 1m 11s
2:	learn: 0.6926973	total: 145ms	remaining: 48.2s
3:	learn: 0.6926113	total: 147ms	remaining: 36.5s
4:	learn: 0.6924752	total: 148ms	remaining: 29.5s
5:	learn: 0.6923470	total: 150ms	remaining: 24.8s
6:	learn: 0.6922307	total: 151ms	remaining: 21.5s
7:	learn: 0.6921232	total: 153ms	remaining: 19s
8:	learn: 0.6920206	total: 155ms	remaining: 17s
9:	learn: 0.6918812	total: 156ms	remaining: 15.5s
10:	learn: 0.6917209	total: 158ms	remaining: 14.2s
11:	learn: 0.6915715	total: 159ms	remaining: 13.1s
12:	learn: 0.6914047	total: 161ms	remaining: 12.2s
13:	learn: 0.6912644	total: 162ms	remaining: 11.4s
14:	learn: 0.6911483	total: 164ms	remaining: 10.8s
15:	learn: 0.6910170	total: 165ms	remaining: 10.2s
16:	learn: 0.6909230	total: 167ms	remaining: 9.65s
17:	learn: 0.6908262	total: 169ms	remaining: 9.19s
18:	learn: 0.6907278	total: 170ms	remaining: 8.78s
19:	learn: 0.

In [10]:
print('Model is fitted: ' + str(cb.is_fitted()))
print('Model params:')
print(cb.get_params())

Model is fitted: True
Model params:
{}


In [11]:
y_pred = cb.predict(X_test)

In [12]:
y_pred

array([0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1,

In [13]:
print('CatBoost Score:', cb.score(X_test, y_test))

CatBoost Score: 0.520066889632107
